import required packages

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image,ImageDraw
import tifffile as tiff
import seaborn as sns
from skimage.measure import label,regionprops
import cv2

list the items present in the train folder

In [ ]:
ls ../input/hubmap-kidney-segmentation/train

list the items in test folder

In [ ]:
ls ../input/hubmap-kidney-segmentation/test/

list the items present in hubmap-kidney-segmentation folder

In [ ]:
ls ../input/hubmap-kidney-segmentation/

read the HuBMAP-20-dataset_information.csv file and make data frame

In [ ]:
TRAIN_PATH = "../input/hubmap-kidney-segmentation/train/"
TEST_PATH = "../input/hubmap-kidney-segmentation/test/"
#info_df = pd.read_csv("../input/hubmap-kidney-segmentation/HuBMAP-20-dataset_information.csv")
#print(info_df.shape)
#(info_df.T)

read the train csv file and make data frame

In [ ]:
train_df = pd.read_csv("../input/hubmap-kidney-segmentation/train.csv")
print(train_df.shape)
train_df.T

In [ ]:
train_df.iloc[4]

In [ ]:
train_df.iloc[4,0]

In [ ]:
train_df.iloc[4,1]

open the 7th index anatomical_structures_segmention_file.json file and load the json data

In [ ]:
#with open(TRAIN_PATH + train_df.iloc[4,0] + "-anatomical-structure.json")as f:
    #data = json.load(f)
#print(data)

read the 7th index tiff image file

In [ ]:
image1 = tiff.imread(TRAIN_PATH + train_df.iloc[4,0] + ".tiff")
print(image1.shape)

plot the kidney image

In [ ]:
image1 = image1[0][0].transpose(1,2,0)
print(image1.shape)
plt.figure(figsize=(10, 10))
plt.imshow(image1)
plt.show()

In [ ]:
## We need to decode the mask from encoding column of train.csv
## https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
def rle2mask(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    print(starts, ends)
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
## Plot all the Glomeruli in this particular kidney
mask1 = rle2mask(train_df.iloc[4, 1], (image1.shape[1], image1.shape[0]))

In [ ]:
## The same kidney image with all the masks
plt.figure(figsize=(10, 10))
plt.imshow(image1)
plt.imshow(mask1, alpha=0.5, cmap='plasma')
plt.show()


In [ ]:
lbl_0 = label(mask1)
props = regionprops(lbl_0)
print(len(props))
bboxes = []
for prop in props:
    bboxes.append([prop.bbox[0] - 30, prop.bbox[1] - 30, 
                   prop.bbox[2] + 30, prop.bbox[3] + 30])
    

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image1[bboxes[0][0]:bboxes[0][2],bboxes[0][1]:bboxes[0][3],:])
plt.imshow(mask1[bboxes[0][0]:bboxes[0][2],bboxes[0][1]:bboxes[0][3]],alpha=0.5,cmap='plasma') 
plt.show()            

In [ ]:
#import the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tifffile as tiff
import cv2
import os
from tqdm.notebook import tqdm
import zipfile

In [ ]:
sz = 256
reduce = 4
MASKS = '../input/hubmap-kidney-segmentation/train.csv'
DATA = '../input/hubmap-kidney-segmentation/train/'
OUT_TRAIN = 'train.zip'
OUT_MASKS = 'masks.zip'


In [ ]:
#functions to convert encoding to mask and mask to encoding
def enc2mask(encs, shape):
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for m,enc in enumerate(encs):
        if isinstance(enc,np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1 + m
    return img.reshape(shape).T

def mask2enc(mask, n=1):
    pixels = mask.T.flatten()
    encs = []
    for i in range(1,n+1):
        p = (pixels == i).astype(np.int8)
        if p.sum() == 0: encs.append(np.nan)
        else:
            p = np.concatenate([[0], p, [0]])
            runs = np.where(p[1:] != p[:-1])[0] + 1
            runs[1::2] -= runs[::2]
            encs.append(' '.join(str(x) for x in runs))
    return encs


In [ ]:
df_masks = pd.read_csv(MASKS).set_index('id')
df_masks.head()

In [ ]:
s_th = 40
p_th = 200*sz//256

x_tot, x2_tot = [],[]
with zipfile.ZipFile(OUT_TRAIN,'w') as img_out, \
zipfile.ZipFile(OUT_MASKS,'w') as mask_out:
    for index,encs in tqdm(df_masks.iterrows(),total=len(df_masks)):
        #read image and generate the mask
        img = tiff.imread(os.path.join(DATA,index+'.tiff'))
        if len(img.shape) == 5:
            img = np.transpose(img.squeeze(),(1,2,0))
            mask = enc2mask(encs,(img.shape[1],img.shape[0]))
            #add padding to make the image dividable into tiles
            shape = img.shape
            pad0 = (reduce*sz - shape[0]%(reduce*sz))%(reduce*sz)
            pad1 = (reduce*sz - shape[1]%(reduce*sz))%(reduce*sz)
            img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=0)
            mask = np.pad(mask,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2]],constant_values=0)
            #split image and mask into tiles using the reshape+transpose trick
            img = cv2.resize(img,(img.shape[1]//reduce,img.shape[0]//reduce),interpolation=cv2.INTER_AREA)
            img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
            img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
            
            mask = cv2.resize(mask,(mask.shape[1]//reduce,mask.shape[0]//reduce),interpolation=cv2.INTER_NEAREST)
            mask = mask.reshape(mask.shape[0]//sz,sz,mask.shape[1]//sz,sz)
            mask = mask.transpose(0,2,1,3).reshape(-1,sz,sz)
            
            for i,(im,m) in enumerate(zip(img,mask)):
                hsv = cv2.cvtColor(im,cv2.COLOR_BGR2HSV)
                h,s,v = cv2.split(hsv)
                if (s>s_th).sum()<=p_th or im.sum() <= p_th:
                    continue
                x_tot.append((im/255.0).reshape(-1,3).mean(0))
                x2_tot.append(((im/255.0)**2).reshape(-1,3).mean(0))
                
                im = cv2.imencode('.png',cv2.cvtColor(im,cv2.COLOR_RGB2BGR))[1]
                img_out.writestr(f'{index}_{i}.png',im)
                m = cv2.imencode('.png',m)[1]
                mask_out.writestr(f'{index}_{i}.png',m)
#image stats
img_avr= np.array(x_tot).mean(0)
img_std = np.sqrt(np.array(x2_tot).mean(0)-img_avr**2)
print('mean:',img_avr, ', std:',img_std)
                
            
            
            
        

In [ ]:
columns, rows = 4,4
idx0 = 30
fig = plt.figure(figsize=(columns*4,rows*4))
with zipfile.ZipFile(OUT_TRAIN,'r') as img_arch, \
zipfile.ZipFile(OUT_MASKS,'r') as msk_arch:
    fnames = sorted(img_arch.namelist())[8:]
    for i in range(rows):
        for j in range(columns):
            idx = i+j*columns
            img = cv2.imdecode(np.frombuffer(img_arch.read(fnames[idx0+idx]),np.uint8),cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
            
            mask = cv2.imdecode(np.frombuffer(msk_arch.read(fnames[idx0+idx]),np.uint8),cv2.IMREAD_GRAYSCALE)
            fig.add_subplot(rows,columns,idx+1)
            plt.axis('off')
            plt.imshow(Image.fromarray(img))
            plt.imshow(Image.fromarray(mask),alpha=0.2)
plt.show()

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision.all import *
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
import cv2
import gc
import random
from albumentations import *
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from lovasz import lovasz_hinge

import warnings
warnings.filterwarnings("ignore")

In [ ]:
bs = 64
nfolds = 4
fold = 0
SEED = 2021
TRAIN = '../input/hubmap-256x256/train/'
MASKS = '../input/hubmap-256x256/masks/'
LABELS = '../input/hubmap-kidney-segmentation/train.csv'
NUM_WORKERS = 4

In [ ]:
mean = np.array([0.61081577, 0.40450876, 0.63537308])
std = np.array([0.14212932, 0.22208199, 0.12562794])

def img2tensor(img,dtype:np.dtype=np.float32):
    if img.ndim == 2: 
        img = np.expand_dims(img,2)
    img = np.transpose(img,(2,0,1))
    return torch.from_numpy(img.astype(dtype,copy=False))

class HuBMAPDataset(Dataset):
    def __init__(self, fold=fold, train=True, tfms=None):
        ids = pd.read_csv(LABELS).id.values
        kf = KFold(n_splits=nfolds,random_state=SEED,shuffle=True)
        ids = set(ids[list(kf.split(ids))[fold][0 if train else 1]])
        self.fnames = [fname for fname in os.listdir(TRAIN) if fname.split('_')[0] in ids]
        self.train = train
        self.tfms = tfms
    
    
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self,idx):
        fname = self.fnames[idx]
        img = cv2.cvtColor(cv2.imread(os.path.join(TRAIN,fname)),cv2.COLOR_BGR2RGB)
        mask = cv2.imread(os.path.join(MASKS,fname),cv2.IMREAD_GRAYSCALE)
        if self.tfms is not None:
            augmented = self.tfms(image=img,mask=mask)
            img,mask = augmented['image'],augmented['mask']
        
        return img2tensor((img/255.0 - mean)/std),img2tensor(mask)
    
def get_aug(p=1.0):
    return Compose([
            HorizontalFlip(),
            VerticalFlip(),
            RandomRotate90(),
            ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.9, border_mode=cv2.BORDER_REFLECT),
            OneOf([
                OpticalDistortion(p=0.3),
                GridDistortion(p=.1),
                IAAPiecewiseAffine(p=0.3),
            ],p=0.3),
            OneOf([
                HueSaturationValue(10,15,10),
                CLAHE(clip_limit=2),
                RandomBrightnessContrast(),
            ],p=0.3),],p=p)

In [ ]:
ds = HuBMAPDataset(tfms=get_aug())
dl = DataLoader(ds,batch_size=64,shuffle=False,num_workers=NUM_WORKERS)
imgs,masks = next(iter(dl))

plt.figure(figsize=(16,16))
for i,(img,mask) in enumerate(zip(imgs,masks)):
    img = ((img.permute(1,2,0)*std+mean)*255.0).numpy().astype(np.uint8)
    plt.subplot(8,8,i+1)
    plt.imshow(img,vmin=0,vmax=255)
    plt.imshow(mask.squeeze().numpy(),alpha=0.2)
    plt.axis('off')
    plt.subplots_adjust(wspace=None,hspace=None)

del ds,dl,imgs,masks

In [ ]:
ind ='e79de561c.tiff' 
img = tiff.imread(os.path.join(DATA,ind))
print(img.shape)
print(len(img.shape))
img = np.transpose(img.squeeze(),(1,2,0))
print(img.shape)
print(len(img.shape))

In [ ]:
(reduce*sz - 16180%(reduce*sz))%(reduce*sz)

In [ ]:
reduce*sz

In [ ]:
pad0 = 204
pad0//2,pad0-pad0//2